需要的信息：entryID\Taxon ID of the species\sequence   OBO中的几个term: Molecular Function (MF), Biological Process (BP), and Cellular Component (CC)
https://github.com/claradepaolis/InformationAccretion (IA文件评分)

In [51]:
import numpy as np
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/home/angel/Temp/Protein/cafa-5-protein-function-prediction'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/home/angel/Temp/Protein/cafa-5-protein-function-prediction/sample_submission.tsv
/home/angel/Temp/Protein/cafa-5-protein-function-prediction/IA.txt
/home/angel/Temp/Protein/cafa-5-protein-function-prediction/Test (Targets)/testsuperset-taxon-list.tsv
/home/angel/Temp/Protein/cafa-5-protein-function-prediction/Test (Targets)/testsuperset.fasta
/home/angel/Temp/Protein/cafa-5-protein-function-prediction/Train/train_taxonomy.tsv
/home/angel/Temp/Protein/cafa-5-protein-function-prediction/Train/go-basic.obo
/home/angel/Temp/Protein/cafa-5-protein-function-prediction/Train/train_terms.tsv
/home/angel/Temp/Protein/cafa-5-protein-function-prediction/Train/train_sequences.fasta


In [52]:
!pip install Bio

In [53]:
#处理/Train/train_sequences.fasta
from Bio import SeqIO

In [54]:
sequences_dict = {}
for seq_record in SeqIO.parse('../cafa-5-protein-function-prediction/Train/train_sequences.fasta', "fasta"):
    sequences_dict[seq_record.id] = str(seq_record.seq)
sequences = pd.DataFrame(list(sequences_dict.items()), columns=['EntryID', 'Sequence'])

In [55]:
#处理/Train/train_annotations.csv
sequences_annotations = pd.read_csv('../cafa-5-protein-function-prediction/Train/train_taxonomy.tsv',sep='\t')

In [56]:
#把两个表合并
train = pd.merge(sequences, sequences_annotations, on='EntryID')

In [57]:
#使用obonet处理/Train/go-basic.obo
!pip install obonet

In [58]:
!pip show pandas

Name: pandas
Version: 1.4.4
Summary: Powerful data structures for data analysis, time series, and statistics
Home-page: https://pandas.pydata.org
Author: The Pandas Development Team
Author-email: pandas-dev@python.org
License: BSD-3-Clause
Location: /home/angel/anaconda3/envs/pytorch2/lib/python3.9/site-packages
Requires: numpy, python-dateutil, pytz
Required-by: bio, d2l, ontobio


In [59]:
import obonet
import pandas as pd

# 读取OBO文件并构建网络图
def read_obo(file_path):
    return obonet.read_obo(file_path)

# 提取指定命名空间的term并返回DataFrame
def extract_terms(graph, namespaces):
    terms_list = []
    for node, data in graph.nodes(data=True):
        if 'namespace' in data and data['namespace'] in namespaces:
            terms_list.append({
                'ID': node,
                'Name': data['name'],
                'Namespace': data['namespace']
            })
    return pd.DataFrame(terms_list)

# 主程序
def main(par):
    # 读取OBO文件并构建网络图
    graph = read_obo(par)
    # 提取GO ID和指定命名空间的term
    namespaces = {"molecular_function", "biological_process", "cellular_component"}
    terms_df = extract_terms(graph, namespaces)
    terms_df.columns = ['term', 'Name', 'Namespace']  # 重命名列
    return terms_df

if __name__ == '__main__':
    par = "../cafa-5-protein-function-prediction/Train/go-basic.obo"
    obo_info = main(par)
    print(obo_info)


             term                                               Name  \
0      GO:0000001                          mitochondrion inheritance   
1      GO:0000002                   mitochondrial genome maintenance   
2      GO:0000003                                       reproduction   
3      GO:0000006  high-affinity zinc transmembrane transporter a...   
4      GO:0000007  low-affinity zinc ion transmembrane transporte...   
...           ...                                                ...   
43243  GO:2001313  UDP-4-deoxy-4-formamido-beta-L-arabinopyranose...   
43244  GO:2001314  UDP-4-deoxy-4-formamido-beta-L-arabinopyranose...   
43245  GO:2001315  UDP-4-deoxy-4-formamido-beta-L-arabinopyranose...   
43246  GO:2001316                       kojic acid metabolic process   
43247  GO:2001317                    kojic acid biosynthetic process   

                Namespace  
0      biological_process  
1      biological_process  
2      biological_process  
3      molecular_functi

In [60]:
#读入train_terms.tsv
sequences_terms = pd.read_csv('../cafa-5-protein-function-prediction/Train/train_terms.tsv',sep='\t')

In [61]:
obo_info2=pd.merge(obo_info, sequences_terms, on='term')

In [64]:
#这个是最终的数据表
train_total=pd.merge(train, obo_info2, on='EntryID').iloc[:,:-1]